In [1]:
import pandas as pd
import datetime as dt
import plotly.graph_objects as go
from technicals.indicators import *
from technicals.patterns import apply_patterns
from technicals.patternsInRange import apply_patterns_in_range

from guruTester import GuruTester,GuruTester2
from plotting import CandlePlot
import MetaTrader5 as mt5
from datetime import datetime
from getCandle import Rates
from sklearn.neural_network import MLPClassifier
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler  
import pickle

In [2]:
pair = "GBPUSD"


In [3]:
mt5.initialize()

True

In [4]:
to_date = datetime.now()
from_date= datetime(2022,8,3,0,0)
rates = mt5.copy_rates_range(pair, mt5.TIMEFRAME_H1,from_date,to_date)
df_an = pd.DataFrame(rates)
df_an["time"] = pd.to_datetime(df_an["time"], unit="s")
df_an

,time,open,high,low,close,tick_volume,spread,real_volume
0,2022-08-02 21:00:00,1.21901,1.21930,1.21659,1.21677,10659,10,0
1,2022-08-02 22:00:00,1.21678,1.21788,1.21656,1.21668,8978,10,0
2,2022-08-02 23:00:00,1.21668,1.21688,1.21574,1.21620,1696,10,0
3,2022-08-03 00:00:00,1.21603,1.21653,1.21592,1.21614,20046,21,0
4,2022-08-03 01:00:00,1.21612,1.21723,1.21560,1.21590,5514,10,0
...,...,...,...,...,...,...,...,...
3979,2023-03-24 18:00:00,1.22233,1.22309,1.22191,1.22194,7417,0,0
3980,2023-03-24 19:00:00,1.22194,1.22310,1.22126,1.22287,6085,0,0
3981,2023-03-24 20:00:00,1.22287,1.22300,1.22193,1.22272,4723,0,0
3982,2023-03-24 21:00:00,1.22272,1.22342,1.22235,1.22259,2957,0,0


In [5]:
# rates = Rates(pair, NB_H1_CANDLES, mt5.TIMEFRAME_H1)
# SPREAD = rates.get_spread()
# df_an = rates.get_rates_from_now()
# df_an.drop(NB_H1_CANDLES-1,inplace=True) 
df_an.drop(['tick_volume', 'spread', 'real_volume'], axis=1, inplace=True)
SPREAD  = 1e-4

In [6]:
candle_patterns = ['HANGING_MAN', 'SHOOTING_STAR',
       'SPINNING_TOP', 'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP',
       'TWEEZER_BOTTOM', 'MORNING_STAR', 'EVENING_STAR']
df_an = apply_patterns_in_range(df_an)
for cp in candle_patterns:
    df_an[cp] = df_an[cp].astype(float)
    
df_an = BollingerBandsFeature(df_an)
df_an = ATRFeature(df_an)
df_an = KeltnerChannelsFeature(df_an)
df_an = RSIFeature(df_an)
df_an = MACDFeature(df_an)
df_an['pivots_l'] = False
df_an['pivots_h'] = False

df_an['low_perv'] = df_an.low.shift(1)
df_an['high_perv'] = df_an.high.shift(1)
df_an['low_next'] = df_an.low.shift(-1)
df_an['high_next'] = df_an.high.shift(-1)
df_an['pivots_l'] = (df_an.low_perv > df_an.low) & (df_an.low_next > df_an.low)
df_an['pivots_h'] = (df_an.high_perv < df_an.high) & (df_an.high_next < df_an.high)
df_an.dropna(inplace=True)
df_an.reset_index(drop=True, inplace=True)

df_an.pivots_l = df_an.pivots_l.astype(float)
df_an.pivots_h = df_an.pivots_h.astype(float)

In [7]:
X = []
Y = [] 
feature_col = [
#     'open', 'high', 'low', 'close',
#                'RSI_14', 'ATR_14',
#     'BB_MA', 'BB_UP','BB_LW', 'EMA', 'KeUp', 'KeLo', 
#     'MACD', 'SIGNAL_MACD', 'HIST',
    
    'Feat_BB_MA_c', 'Feat_BB_UP_c', 'Feat_BB_LW_c',
    'Feat_EMA_c', 'Feat_KeUp_c', 'Feat_KeLo_c',
    
    'Feat_BB_MA_o', 'Feat_BB_UP_o', 'Feat_BB_LW_o',
    'Feat_EMA_o', 'Feat_KeUp_o', 'Feat_KeLo_o', 
    
    'Feat_BB_MA_l', 'Feat_BB_UP_l', 'Feat_BB_LW_l',
    'Feat_EMA_l', 'Feat_KeUp_l', 'Feat_KeLo_l',
    
    'Feat_BB_MA_h', 'Feat_BB_UP_h', 'Feat_BB_LW_h',
    'Feat_EMA_h', 'Feat_KeUp_h', 'Feat_KeLo_h',
    
    'Feat_ATR_14',
    'Feat_gains','Feat_wins_rma', 'Feat_losses_rma', 'Feat_RSI_14', 
#     'Feat_MACD', 'Feat_SIGNAL_MACD', 'Feat_HIST',
    
        'full_range',# this is added ...
       'body_lower', 'body_upper', 
        'body_bottom_perc', 'body_top_perc',
       'body_perc', 'direction', 'body_size', 'low_change', 'high_change',
    'mid_point', 'mid_point_prev_2', 'body_size_prev',
       'direction_prev', 'direction_prev_2', 'body_perc_prev','body_perc_prev_2',
        'HANGING_MAN', 'SHOOTING_STAR', 'SPINNING_TOP',
       'MARUBOZU', 'ENGULFING',
    'TWEEZER_TOP', 'TWEEZER_BOTTOM',
       'MORNING_STAR', 'EVENING_STAR'
]

In [8]:
indeces = []
for i in range(4,len(df_an)):
#     if df_an.pivots_h[i-1] or df_an.pivots_l[i-1] :
    feature = []
    feature.append(df_an.pivots_h[i-1])
    feature.append(df_an.pivots_l[i-1])
    feature.append(df_an.pivots_h[i-2])
    feature.append(df_an.pivots_l[i-2])
    feature.append(df_an.pivots_h[i-3])
    feature.append(df_an.pivots_l[i-3])
    feature.append(df_an.pivots_h[i-4])
    feature.append(df_an.pivots_l[i-4])
    for fc in feature_col:
        feature.append(df_an[fc][i])
        feature.append(df_an[fc][i-1])
        feature.append(df_an[fc][i-2])
        feature.append(df_an[fc][i-3])
        feature.append(df_an[fc][i-4])

    X.append(feature)
#         Y.append(df_an.SIGNAL[i])
    indeces.append(i)
    
X = np.array(X)

In [9]:
coefs_0 = np.genfromtxt('coefs_0.csv', delimiter=',')
coefs_1 = np.genfromtxt('coefs_1.csv', delimiter=',')
coefs_2 = np.genfromtxt('coefs_2.csv', delimiter=',')
intercepts_0 = np.genfromtxt('intercepts_0.csv', delimiter=',')
intercepts_1 = np.genfromtxt('intercepts_1.csv', delimiter=',')
intercepts_2 = np.genfromtxt('intercepts_2.csv', delimiter=',')
coefs_0.shape

(283, 200)

In [10]:
def softmax(x):
#     return np.exp(x ) / np.exp(x.sum())
    return(np.exp(x - np.max(x)) / np.exp(x - np.max(x)).sum())

def relu(x):
    return (abs(x) + x) / 2


In [11]:
l1 = np.matmul(X,coefs_0) + intercepts_0
l1 = relu(l1)
l2 = np.matmul(l1, coefs_1) + intercepts_1
l2 = relu(l2)
l3 = np.matmul(l2, coefs_2) + intercepts_2
l3 = softmax(l3)
pred = np.argmax(l3,axis=1)-1
# l3.shape

In [12]:
# rates = mt5.copy_rates_range(pair, mt5.TIMEFRAME_M5,from_date,to_date)
# df_m5 = pd.DataFrame(rates)
# df_m5["time"] = pd.to_datetime(df_m5["time"], unit="s")
# df_m5.reset_index(drop=True, inplace=True)
# df_m5.drop(['tick_volume', 'spread', 'real_volume'], axis=1, inplace=True)
# df_m5

In [13]:
rates = Rates(pair, 50000, mt5.TIMEFRAME_M5)
df_m5 = rates.get_rates_from_now()
df_m5.reset_index(drop=True, inplace=True)
df_m5.drop(['tick_volume', 'spread', 'real_volume'], axis=1, inplace=True)
df_m5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   time    50000 non-null  datetime64[ns]
 1   open    50000 non-null  float64       
 2   high    50000 non-null  float64       
 3   low     50000 non-null  float64       
 4   close   50000 non-null  float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 1.9 MB


In [14]:

df_test = df_an[['time', 'open', 'high', 'low', 'close','pivots_l','pivots_h']].copy()


In [15]:
df_test['SL'] = 0
df_test['TP'] = 0 
df_test['SIGNAL'] = 0
for i in range(4,len(df_test)):
#     if df_test.pivots_h[i-1] or df_test.pivots_l[i-1]:
    if df_test.pivots_l[i-1]==1:
        if(pred[i-4] == 1.0):
            df_test.SIGNAL[i] = pred[i-4]
            df_test['SL'][i] = df_test.low[i-1] 
            df_test['TP'][i] = 2*(df_test.close[i] - df_test.low[i-1]) + df_test.close[i]
    if df_test.pivots_h[i-1]==1:
        if (pred[i-4]== -1.0 ):
            df_test.SIGNAL[i] = pred[i-4]
            df_test['SL'][i] = df_test.high[i-1]
            df_test['TP'][i] = 2*(df_test.close[i] - df_test.high[i-1] ) + df_test.close[i]

C:\Users\meysam\AppData\Local\Temp\ipykernel_6296\597360150.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test.SIGNAL[i] = pred[i-4]
C:\Users\meysam\AppData\Local\Temp\ipykernel_6296\597360150.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['SL'][i] = df_test.low[i-1]
C:\Users\meysam\AppData\Local\Temp\ipykernel_6296\597360150.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['TP'][i] = 2*(df_test.close[

In [25]:
df_test[df_test.SIGNAL != 0]

,time,open,high,low,close,pivots_l,pivots_h,SL,TP,SIGNAL
9,2022-08-04 15:00:00,1.20849,1.21196,1.20711,1.21118,0.0,0.0,1.20649,1.22056,1
26,2022-08-05 08:00:00,1.21398,1.21489,1.21363,1.21439,0.0,0.0,1.21356,1.21605,1
32,2022-08-05 14:00:00,1.21403,1.21465,1.21352,1.21385,0.0,0.0,1.21240,1.21675,1
35,2022-08-05 17:00:00,1.20483,1.20694,1.20353,1.20615,0.0,1.0,1.20024,1.21797,1
39,2022-08-05 21:00:00,1.20726,1.20770,1.20610,1.20632,0.0,0.0,1.20933,1.20030,-1
...,...,...,...,...,...,...,...,...,...,...
3914,2023-03-23 10:00:00,1.23105,1.23286,1.23045,1.23206,1.0,0.0,1.23434,1.22750,-1
3919,2023-03-23 15:00:00,1.22753,1.23184,1.22732,1.23148,0.0,0.0,1.23375,1.22694,-1
3937,2023-03-24 09:00:00,1.22675,1.22883,1.22675,1.22710,0.0,1.0,1.22666,1.22798,1
3945,2023-03-24 17:00:00,1.22245,1.22293,1.22132,1.22234,0.0,0.0,1.22319,1.22064,-1


In [21]:
gt = GuruTester2(
        df_test,
        df_m5,
        1e-4,
        use_spread=True
    )
df_res_m5 = gt.run_test()

prepare_data...
run_test...
Result:
win rate:  0.33285302593659943
-1.0    463
 2.0    231
Name: result, dtype: int64


In [22]:
df_res_m5[df_res_m5.SIGNAL != 0]

,running,start_index_m5,profit_factor,loss_factor,SPREAD,start_price,trigger_price,SIGNAL,TP,SL,result,end_time,start_time
0,False,3129,2,-1.0,0.0001,1.21449,1.21355,1,1.21605,1.21356,-1.0,2022-08-05 09:10:00,2022-08-05 09:00:00
1,False,3201,2,-1.0,0.0001,1.21395,1.20692,1,1.21675,1.21240,-1.0,2022-08-05 15:30:00,2022-08-05 15:00:00
2,False,2926,2,-1.0,0.0001,1.21128,1.20648,1,1.22056,1.20649,-1.0,2022-08-05 15:40:00,2022-08-04 16:00:00
3,False,3342,2,-1.0,0.0001,1.20611,1.20531,1,1.20685,1.20559,-1.0,2022-08-08 03:00:00,2022-08-08 03:00:00
4,False,3318,2,-1.0,0.0001,1.20645,1.20778,-1,1.20407,1.20764,-1.0,2022-08-08 05:50:00,2022-08-08 01:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
689,False,49570,2,-1.0,0.0001,1.23206,1.22747,-1,1.22750,1.23434,2.0,2023-03-23 14:45:00,2023-03-23 11:00:00
690,False,49630,2,-1.0,0.0001,1.23148,1.22624,-1,1.22694,1.23375,2.0,2023-03-23 20:55:00,2023-03-23 16:00:00
691,False,49845,2,-1.0,0.0001,1.22720,1.22421,1,1.22798,1.22666,-1.0,2023-03-24 10:05:00,2023-03-24 10:00:00
692,False,49941,2,-1.0,0.0001,1.22234,1.22346,-1,1.22064,1.22319,-1.0,2023-03-24 21:20:00,2023-03-24 18:00:00


In [19]:
df_test[df_test.SIGNAL != 0]


,time,open,high,low,close,pivots_l,pivots_h,SL,TP,SIGNAL
9,2022-08-04 15:00:00,1.20849,1.21196,1.20711,1.21118,0.0,0.0,1.20649,1.22056,1
26,2022-08-05 08:00:00,1.21398,1.21489,1.21363,1.21439,0.0,0.0,1.21356,1.21605,1
32,2022-08-05 14:00:00,1.21403,1.21465,1.21352,1.21385,0.0,0.0,1.21240,1.21675,1
35,2022-08-05 17:00:00,1.20483,1.20694,1.20353,1.20615,0.0,1.0,1.20024,1.21797,1
39,2022-08-05 21:00:00,1.20726,1.20770,1.20610,1.20632,0.0,0.0,1.20933,1.20030,-1
...,...,...,...,...,...,...,...,...,...,...
3914,2023-03-23 10:00:00,1.23105,1.23286,1.23045,1.23206,1.0,0.0,1.23434,1.22750,-1
3919,2023-03-23 15:00:00,1.22753,1.23184,1.22732,1.23148,0.0,0.0,1.23375,1.22694,-1
3937,2023-03-24 09:00:00,1.22675,1.22883,1.22675,1.22710,0.0,1.0,1.22666,1.22798,1
3945,2023-03-24 17:00:00,1.22245,1.22293,1.22132,1.22234,0.0,0.0,1.22319,1.22064,-1
